# This file is the full pipeline to run SVM Modelling
### Clone the whole repo and keep this pipeline internal to the svm_one_beat folder
Note: Waveform Data folder (with patient data) will either need to be moved into a folder and will need to provided as an absolute path since it is no longer internal to the folder.

In [1]:
#Imports

from data_loading import load_label_event
from svm_model import build_svm
from svm_leave_one_out import build_svm_leave_one_out
import os
import h5py
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd
import matplotlib.pyplot as plt
import csv

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC

from sklearn.decomposition import PCA
from mlxtend.plotting import plot_decision_regions

In [14]:
#Load Data
# 1. change first input to Local path to Waveform Data folder
# 2. changed second input to local path to Labelled_Events.xlsx file
# 4. Local folder to save processed data output (csv format) to
svp = 'Modeling Data' #path to save dataset for modeling too
load_label_event('Waveform Data', 'Labelled_Events.xlsx', excel_sheet_name='Summary', save_file_path=svp)


After detrend before wavelet:  [146.67326 167.97446 173.65193 ... 172.30809 172.17563 170.23831]
After detrend before wavelet:  [136.20001 136.20001 136.2     ... 136.20001 136.20001 136.20001]


/opt/anaconda3/lib/python3.8/site-packages/scipy/signal/signaltools.py:1475: RuntimeWarning: divide by zero encountered in true_divide
  res *= (1 - noise / lVar)
/opt/anaconda3/lib/python3.8/site-packages/scipy/signal/signaltools.py:1475: RuntimeWarning: invalid value encountered in multiply
  res *= (1 - noise / lVar)
/Users/richard.morse/Downloads/TCH_Arrhythmias_F20-master/SVM_one_beat/PPG_feature_extraction.py:20: PeakPropertyWarning: some peaks have a width of 0
  half_width = signal.peak_widths(ppg_signal, peaks, rel_height=0.5)[0]  # peak width at half maximum
/Users/richard.morse/Downloads/TCH_Arrhythmias_F20-master/SVM_one_beat/PPG_feature_extraction.py:21: PeakPropertyWarning: some peaks have a width of 0
  two_thirds = signal.peak_widths(ppg_signal, peaks, rel_height=1 / 3)[0]  # peak width at two thirds of maximum


After detrend before wavelet:  [170.54337 170.51608 170.65344 ... 185.02629 178.43005 178.43396]
After detrend before wavelet:  [159.04759 159.01682 159.32599 ... 165.23798 161.90347 161.95839]
After detrend before wavelet:  [122.39794 118.31673 117.96681 ... 116.19998 133.79999 133.79999]
After detrend before wavelet:  [111.09449  111.161156 108.27493  ... 139.28075  138.16217  136.90793 ]
After detrend before wavelet:  [122.49183  120.431915 119.7433   ... 206.36552  206.35887  206.35226 ]
After detrend before wavelet:  [177.63507 177.48822 177.31995 ... 183.81573 183.80495 183.85445]
After detrend before wavelet:  [ 98.33564   98.336876 109.67025  ... 161.69458  162.47438  163.19995 ]
After detrend before wavelet:  [ 93.54817  92.81331  92.5943  ... 256.0112  256.0112  256.0112 ]
After detrend before wavelet:  [152.48279 152.55327 153.85344 ... 156.37167 156.44107 156.50331]
After detrend before wavelet:  [186.22827 186.24611 183.62283 ... 174.20705 174.60179 174.99469]
After detren

After detrend before wavelet:  [274.0161   273.92587  274.2631   274.48575  272.99384  273.1326
 273.30484  273.46808  273.34915  273.30585  273.39917  273.4559
 265.47598  265.50592  265.5629   265.61264  265.6551   265.69952
 265.63882  265.6063   259.20193  259.19     259.2063   259.21503
 259.21616  259.21933  259.2507   259.2745   243.29077  243.30908
 243.3198   243.33257  243.34735  243.36162  243.30806  243.27272
 233.65553  233.63345  233.62953  233.62073  233.60712  233.5948
 233.60063  233.6016   209.59773  209.59515  209.58766  209.5815
 209.57663  209.57143  209.58435  209.59242  198.39558  198.40009
 198.39973  198.4007   185.60295  185.60486  185.6019   185.60025
 169.5999   169.59923  169.59982  169.6001   169.59999  169.6
 169.6      169.6      158.4      158.40001  158.4      158.4
 158.4      158.40001  158.4      158.40001  145.60002  145.60002
 145.60002  145.60004  145.60004  145.60004  145.60002  145.60004
 136.       136.       136.       136.       136.       1

After detrend before wavelet:  [175.9399  175.82451 176.32729 176.66443 176.83594 177.05185 177.10207
 177.1967  158.13568 158.2628  158.2243  158.23016 158.2804  158.31876
 158.40149 158.47235 145.73131 145.79344 145.68994 145.63084 145.61609
 145.58946 145.60721 145.61307 134.40701 134.40416 134.44568 134.47533
 134.49309 134.51404 134.52306 134.53528 134.5507  134.56526 134.51054
 134.4744  134.4568  134.43425 134.43025 134.42128 169.60733 169.59473
 169.60068 169.60165 169.59766 169.59502 169.58734 169.58102 183.97607
 183.97076 183.98398 183.99223 183.99551 184.00015 183.99977 184.00073
 227.20305 227.20503 227.20198 227.20029 227.19995 227.19925 227.19983
 227.20007 233.60002 233.6     233.6     233.6     233.60002 233.60007
 233.6     233.6     268.8     268.79996 268.8     268.80002 268.80002
 268.80005 268.89032 268.95642 269.9644  270.71997 270.3034  270.20096
 270.41257 270.54004 270.42963 270.38293 265.60007 265.60004 265.60004
 265.6     265.60004 265.60004 265.60007 265.6

...

After detrend before wavelet:  [115.40334  110.864716 110.52713  ... 250.84528  250.4754   250.30356 ]
After detrend before wavelet:  [201.6132  194.51321 191.15561 ... 156.4827  156.1074  155.77733]
After detrend before wavelet:  [219.47237 219.34584 219.32439 ... 238.28908 238.35614 238.39365]
After detrend before wavelet:  [171.25798 171.03131 171.27664 ... 249.60007 249.60005 249.60004]
After detrend before wavelet:  [311.78845 311.57733 310.99634 ... 269.316   269.29907 269.2732 ]
After detrend before wavelet:  [162.26059 200.48413 201.20615 ... 180.27057 237.85132 237.83148]
After detrend before wavelet:  [290.15323 289.6661  289.29477 ... 144.82356 144.92691 145.10526]
After detrend before wavelet:  [223.367   224.14581 220.55133 ... 165.25229 164.44113 163.73668]
After detrend before wavelet:  [212.06755 211.98322 212.10541 ... 197.00479 196.55228 196.10855]
After detrend before wavelet:  [197.06218 196.98395 197.41551 ... 228.80647 228.80664 228.80682]
After detrend before wav

After detrend before wavelet:  [90.24187 90.25982 90.09523 ... 89.05325 89.19731 89.36041]
After detrend before wavelet:  [276.2899  276.14624 276.9664  ... 137.43222 137.5378  129.59795]
After detrend before wavelet:  [209.46742 209.2511  209.95882 ... 246.00519 246.70221 247.39539]
After detrend before wavelet:  [125.181854 125.26689  125.05583  ... 185.53635  185.53308  185.52985 ]
After detrend before wavelet:  [211.50757 211.50218 211.51053 ... 240.08463 239.91037 239.785  ]
After detrend before wavelet:  [275.87775  275.8004   276.31927  276.67838  276.87772  277.11984
 277.20227  267.72745  267.89548  268.05203  268.0377   268.06915
 268.14642  268.21143  268.3222   260.42075  260.50696  260.5965
 260.5301   260.50546  260.52258  260.52853  260.57626  246.21278
 246.23819  246.26656  246.33665  246.39558  246.44333  246.4941
 246.53362  252.97615  253.02168  253.06644  252.85211  252.70721
 252.63177  252.53772  252.51303  214.06978  214.00789  213.951
 214.2026   214.37157  214

After detrend before wavelet:  [175.3828  175.37805 175.76346 ... 218.14578 218.12405 218.0999 ]
After detrend before wavelet:  [261.84894 261.77234 261.8506  ... 222.01442 223.25487 224.10239]
After detrend before wavelet:  [134.85918 134.83066 133.81256 ... 176.19865 175.76852 175.28014]
After detrend before wavelet:  [145.60323  121.704735 120.97357  ... 155.86276  136.68874  136.70848 ]
After detrend before wavelet:  [135.51732  108.321465 108.462234 ... 141.50096  140.83492  140.38808 ]
After detrend before wavelet:  [137.51096 137.4986  137.21184 ... 245.21545 245.16225 245.15018]
After detrend before wavelet:  [141.19513 141.26822 141.68866 ... 257.99985 259.68198 257.93082]
After detrend before wavelet:  [248.83588 248.54599 249.30075 ... 140.5167  139.46445 138.90474]
After detrend before wavelet:  [149.25296 138.12411 138.05307 ... 289.19153 289.20236 294.01056]
After detrend before wavelet:  [187.33705 176.43958 176.17056 ... 161.44147 161.39954 161.34692]
After detrend befo

After detrend before wavelet:  [338.61856 338.59988 338.0174  ... 260.7731  260.7965  260.89795]
After detrend before wavelet:  [179.72624 179.42273 179.16031 ... 147.5308  147.43129 147.42876]
After detrend before wavelet:  [142.02335 141.94354 140.5422  ... 137.31735 136.73843 136.19968]
After detrend before wavelet:  [158.00955 163.4057  163.48477 ... 149.49377 149.42354 149.378  ]
After detrend before wavelet:  [158.16989 157.22093 156.97469 ... 160.60136 160.60136 160.6013 ]
After detrend before wavelet:  [179.0707   179.07031  179.09804  179.11824  185.93092  185.94559
 185.92104  185.90701  188.10349  188.09715  188.17517  188.2306
 191.46346  191.50232  191.51857  191.5409   191.56929  191.59605
 191.53871  191.50392  189.29167  189.27339  189.27762  189.27583
 188.068    188.06175  188.0781   188.08841  185.89262  185.89848
 185.8983   185.89977  181.5028   181.50545  181.35689  181.24886
 177.7813   177.70291  177.65245  177.59454  176.5523   176.50589
 176.48015  176.44888  

After detrend before wavelet:  [180.71957 180.82858 181.17911 ... 209.17618 209.56375 209.7341 ]
After detrend before wavelet:  [241.74686 241.74689 241.74596 ... 195.2     195.19998 195.2    ]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
After detrend before wavelet:  [182.8 182.8 182.8 ... 182.8 182.8 182.8]
Aft

In [3]:
#Clean Data
#dataset_for_modeling.csv name is preset into data_loading code. Can be changed within that file if desired.
data_path = svp + '/dataset_for_modeling.csv'
sheet_name = "dataset_for_modeling"
feature_list = [ 'pre_R' ,'local_R' ,'post_R' ,'st' ,'dt' ,'half_width' ,'two_thirds'] 

data = pd.read_csv(data_path, header=0, low_memory=False)
for index, row in data.iterrows():
    print('Current row',index)
    if not ('[]' in row['pre_R']):  #data is not missing, convert [] to float number
        if not isinstance(row['pre_R'], float):
            data.loc[index, 'pre_R'] = float(row['pre_R'])
    else:
        data.loc[index, 'pre_R'] = 0 #missing data, fill with 0

    if not ('[]' in row['post_R']):  
        if not isinstance(row['post_R'], float):
            data.loc[index, 'post_R'] = float(row['post_R'])
    else:
        data.loc[index, 'post_R'] = 0

    if not ('[]' in row['local_R']):
        if not isinstance(row['local_R'], float):
            data.loc[index, 'local_R'] = float(row['pre_R'])
    else:
        data.loc[index, 'local_R'] = 0

    if not ('[]' in row['st']):
        # Need to trim off the bracket elements and convert it to a float
        if not isinstance(row['st'], float):
            row['st'] = row['st'][1:-1]
            data.loc[index, 'st'] = float(row['st'])
    else:
        data.loc[index, 'st'] = 0

    if not ('[]' in row['half_width']):
        # Need to trim off the bracket elements and convert it to a float
        if not isinstance(row['half_width'], float):
            row['half_width'] = row['half_width'][1:-1]
            data.loc[index, 'half_width'] = float(row['half_width'][0])
    else:
        data.loc[index, 'half_width'] = 0

    if not ('[]' in row['two_thirds']):
        # Need to trim off the bracket elements and convert it to a float
        if not isinstance(row['two_thirds'], float):
            row['two_thirds'] = row['two_thirds'][1:-1]
            data.loc[index, 'two_thirds'] = float(row['two_thirds'][0])
    else:
        data.loc[index, 'two_thirds'] = 0

    if not ('[]' in row['dt']):
        if not isinstance(row['dt'], float):
            row['dt'] = row['dt'][1:-1]
            data.loc[index, 'dt'] = float(row['dt'])
    else:
        data.loc[index, 'dt'] = 0

    wavelet_features = ['one_beat_ecg_rr_wt_dpb_' + str(i) for i in range(13, 28)]
    for feature in wavelet_features:
        data.loc[index, feature] = float(row[feature])
clean_data_path = svp + '/colab.csv'
f = open(clean_data_path,'w')
data.to_csv(f)

Current row 0
Current row 1
Current row 2
Current row 3
Current row 4
Current row 5
Current row 6
Current row 7
Current row 8
Current row 9
Current row 10
Current row 11
Current row 12
Current row 13
Current row 14
Current row 15
Current row 16
Current row 17
Current row 18
Current row 19
Current row 20
Current row 21
Current row 22
Current row 23
Current row 24
Current row 25
Current row 26
Current row 27
Current row 28
Current row 29
Current row 30
Current row 31
Current row 32
Current row 33
Current row 34
Current row 35
Current row 36
Current row 37
Current row 38
Current row 39
Current row 40
Current row 41
Current row 42
Current row 43
Current row 44
Current row 45
Current row 46
Current row 47
Current row 48
Current row 49
Current row 50
Current row 51
Current row 52
Current row 53
Current row 54
Current row 55
Current row 56
Current row 57
Current row 58
Current row 59
Current row 60
Current row 61
Current row 62
Current row 63
Current row 64
Current row 65
Current row 66
Curre

Current row 519
Current row 520
Current row 521
Current row 522
Current row 523
Current row 524
Current row 525
Current row 526
Current row 527
Current row 528
Current row 529
Current row 530
Current row 531
Current row 532
Current row 533
Current row 534
Current row 535
Current row 536
Current row 537
Current row 538
Current row 539
Current row 540
Current row 541
Current row 542
Current row 543
Current row 544
Current row 545
Current row 546
Current row 547
Current row 548
Current row 549
Current row 550
Current row 551
Current row 552
Current row 553
Current row 554
Current row 555
Current row 556
Current row 557
Current row 558
Current row 559
Current row 560
Current row 561
Current row 562
Current row 563
Current row 564
Current row 565
Current row 566
Current row 567
Current row 568
Current row 569
Current row 570
Current row 571
Current row 572
Current row 573
Current row 574
Current row 575
Current row 576
Current row 577
Current row 578
Current row 579
Current row 580
Current 

In [6]:
#Train model (This will take a long time on a normal machine)
#data_path is clean_data_path as set in last block which is: clean_data_path = svp + '/colab.csv'
feature_list = ['pre_R','local_R','post_R','st','dt','half_width','two_thirds']

wavelet_feature_names = ['one_beat_ecg_rr_wt_dpb_' + str(i) for i in range(13, 28)]

feature_list.extend(wavelet_feature_names)

#for leave one out change to: acc,fals = build_svm_leave_one_out(data_path, feature_list)
model,acc,fals = build_svm(clean_data_path, feature_list)
#model info can be parsed using sklearn SVC commands

print('The accuracy is', acc)
print('False positive rate is', fals)

Unique patients {1}
Accuracy:  1.0
Proportion of Arrhythmias that we accurately detect:  1.0
0 negatives in test set
0
The accuracy is 1.0
False positive rate is 0
